# Complete Example
This notebook contains a complete example of feature importance for a trained neural network

## Imports

In [1]:
import random
import math
import warnings

from typing import List, Dict
from statistics import mean, stdev
from random import randint

import pandas as pd
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
from tensorflow.keras.optimizers import Adam
from IPython.display import HTML
from scipy.stats import norm

warnings.filterwarnings("ignore")

matplotlib.rcParams["animation.embed_limit"] = 2**128
matplotlib.rcParams["figure.dpi"] = 300

## Generating data

In [2]:
samples = 1000
split = 100


def generate_random_values():
    return [randint(0, 50) * randint(0, 50) for _ in range(samples)]


ms = [2 for _ in range(samples)]
xs = generate_random_values()
cs = [50 for _ in range(samples)]

ys = []
for i in range(samples):
    y = ms[i] * xs[i] + cs[i]
    ys.append(y)
noise = [random.randint(0, 100) for _ in range(samples)]

data = {"ms": ms, "xs": xs, "cs": cs, "noise": noise, "ys": ys}

df = pd.DataFrame(data=data)
train = df.iloc[split:]
test = df.iloc[:split]

In [3]:
X_train = train[["ms", "xs", "cs", "noise"]]
y_train = train["ys"]
X_test = test[["ms", "xs", "cs", "noise"]]
y_test = test["ys"]

In [4]:
def create_history(columns: int, nodes: int) -> Dict:
    return {i:{j: [] for j in range(nodes)} for i in range(columns)}

## Defining Model

In [5]:
FIRST_LAYER = 4

In [6]:
history = create_history(len(X_train.columns), FIRST_LAYER)

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=FIRST_LAYER, input_shape=[4]))
model.add(tf.keras.layers.Dense(units=1))
model.summary()
opt = Adam(0.01)
model.compile(optimizer=opt, loss="mean_squared_error")

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 20        
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


2022-12-02 23:10:51.638640: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-02 23:10:51.638768: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
for epoch in range(1, 501):
    print(f"RUNNING EPOCH {epoch}")
    hist = model.fit(X_train, y_train, epochs=1)
    all_weights = model.layers[0].get_weights()[0].tolist()
    for weights in range(len(all_weights)):
        for weight in range(len(all_weights[weights])):
            history[weights][weight].append(all_weights[weights][weight])

RUNNING EPOCH 1


2022-12-02 23:10:51.784788: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


12/29 [===========>..................] - ETA: 0s - loss: 127839.7734

2022-12-02 23:10:51.987581: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


29/29 [==============================] - 0s 5ms/step - loss: 62457.1641
RUNNING EPOCH 2
29/29 [==============================] - 0s 4ms/step - loss: 4359.5967
RUNNING EPOCH 3
29/29 [==============================] - 0s 4ms/step - loss: 1688.0483
RUNNING EPOCH 4
29/29 [==============================] - 0s 4ms/step - loss: 771.0728
RUNNING EPOCH 5
29/29 [==============================] - 0s 4ms/step - loss: 436.1665
RUNNING EPOCH 6
29/29 [==============================] - 0s 5ms/step - loss: 307.6446
RUNNING EPOCH 7
29/29 [==============================] - 0s 4ms/step - loss: 238.3710
RUNNING EPOCH 8
29/29 [==============================] - 0s 5ms/step - loss: 188.9178
RUNNING EPOCH 9
29/29 [==============================] - 0s 5ms/step - loss: 149.2786
RUNNING EPOCH 10
29/29 [==============================] - 0s 4ms/step - loss: 117.3133
RUNNING EPOCH 11
29/29 [==============================] - 0s 4ms/step - loss: 90.1842
RUNNING EPOCH 12
29/29 [==============================] - 0s 4ms/

## Evaluating Model

In [9]:
res_ms = X_test["ms"].values.tolist()
res_xs = X_test["xs"].values.tolist()
res_cs = X_test["cs"].values.tolist()
res_noise = X_test["noise"].values.tolist()
res_pred = [
    round(i) for i in model.predict(X_test).flatten().tolist()
]
res = {
    "ms": res_ms,
    "xs": res_xs,
    "cs": res_cs,
    "noise": res_noise,
    "ys": y_test,
    "pred": res_pred
}

res = pd.DataFrame(res)
res

4/4 [==============================] - 0s 3ms/step


2022-12-02 23:12:16.137700: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,ms,xs,cs,noise,ys,pred
0,2,24,50,100,98,98
1,2,364,50,58,778,778
2,2,1344,50,35,2738,2738
3,2,264,50,29,578,578
4,2,90,50,46,230,230
...,...,...,...,...,...,...
95,2,1786,50,60,3622,3622
96,2,104,50,19,258,258
97,2,630,50,79,1310,1310
98,2,966,50,53,1982,1982


In [10]:
expected = list(res["ys"].values)
actual = list(res["pred"].values)
accuracies = []
for i in range(len(expected)):
    if expected[i] > actual[i]:
        accuracies.append((actual[i] / expected[i]) * 100)
    elif expected[i] < actual[i]:
        accuracies.append((expected[i] / actual[i]) * 100)
    else:
        accuracies.append(100)
print(f"The model is correct to {round(mean(accuracies), 5)}%")

The model is correct to 100%


## Colors
Setting some nice colors for our following plots

In [11]:
light = "#90CCF4"
darker = "#5DA2D5"
important = "#F3D250"
noise = "#F78888"
other = "#ECECEC"

## Animated Bar Graph
This shows how the weights of each of the 

In [ ]:
frames = 500
fig, ax = plt.subplots(figsize=(16,8))

In [13]:
def run(i):
    ax.cla()
    ax.set_xlabel("Input Column and Node Number")
    ax.set_ylabel("Weight")
    ax.set_ylim(-1.5, 1.5)
    m_one = history[0][0][i]
    m_two = history[0][1][i]
    m_three = history[0][2][i]
    m_four = history[0][3][i]

    x_one = history[1][0][i]
    x_two = history[1][1][i]
    x_three = history[1][2][i]
    x_four = history[1][3][i]

    c_one = history[2][0][i]
    c_two = history[2][1][i]
    c_three = history[2][2][i]
    c_four = history[2][3][i]

    noise_one = history[3][0][i]
    noise_two = history[3][1][i]
    noise_three = history[3][2][i]
    noise_four = history[3][3][i]
    ax.bar(
        [
            "M1",
            "M2",
            "M3",
            "M4",
            "X1",
            "X2",
            "X3",
            "X4",
            "C1",
            "C2",
            "C3",
            "C4",
            "Noise 1",
            "Noise 2",
            "Noise 3",
            "Noise 4",
        ],
        [
            m_one,
            m_two,
            m_three,
            m_four,
            x_one,
            x_two,
            x_three,
            x_four,
            c_one,
            c_two,
            c_three,
            c_four,
            noise_one,
            noise_two,
            noise_three,
            noise_four,
        ],
        color=[
            darker,
            darker,
            darker,
            darker,
            important,
            important,
            important,
            important,
            light,
            light,
            light,
            light,
            noise,
            noise,
            noise,
            noise,
        ],
    )

In [14]:
anim = animation.FuncAnimation(fig, run, frames=frames, interval=50)
HTML(anim.to_html5_video())

## Splitting weights into groups

In [ ]:
def get_groups(index: int, nodes: int) -> List:
    group = []
    for i in range(nodes):
        group.append(history[index][i])
    return group

In [ ]:
ms = get_groups(0, 4)
xs = get_groups(1, 4)
cs = get_groups(2, 4)
noises = get_groups(3, 4)

## Box Plots

In [ ]:
def generate_box_plot(
    ax,
    data: List,
    color: str,
    label: str,
    position_offset: int = 0,
    showfliers: bool = False
):
    ax.set_xlabel("Input Column and Node Number")
    ax.set_ylabel("Weight Values")
    color = {"color": color}
    ax.boxplot(
        data,
        positions=[i + position_offset for i in range(1,5)],
        boxprops=color,
        medianprops=color,
        whiskerprops=color,
        capprops=color,
        flierprops={"markeredgecolor": other},
        showfliers=showfliers,
        labels=[f"{label}{i}" for i in range(1,5)]
    )

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))

generate_box_plot(ax, ms, darker, "M")
generate_box_plot(ax, xs, important, "X", 4)
generate_box_plot(ax, cs, light, "C", 8)
generate_box_plot(ax, noises, noise, "Noise", 12)

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))

generate_box_plot(
    ax,
    ms,
    darker,
    "M",
    showfliers=True
)
generate_box_plot(
    ax,
    xs,
    important,
    "X",
    4,
    showfliers=True
)
generate_box_plot(ax,
    cs, light, "C", 8, showfliers=True)
generate_box_plot(ax,
    noises, noise, "Noise", 12, showfliers=True)

## Standard Deviation

In [ ]:
data = {
    "m_one": ms[0],
    "m_two": ms[1],
    "m_three": ms[2],
    "m_four": ms[3],
    "x_one": xs[0],
    "x_two": xs[1],
    "x_three": xs[2],
    "x_four": xs[3],
    "c_one": cs[0],
    "c_two": cs[1],
    "c_three": cs[2],
    "c_four": cs[3],
    "noise_one": noises[0],
    "noise_two": noises[1],
    "noise_three": noises[2],
    "noise_four": noises[3]
}

columns = [
    "m_one",
    "m_two",
    "m_three",
    "m_four",
    "x_one",
    "x_two",
    "x_three",
    "x_four",
    "c_one",
    "c_two",
    "c_three",
    "c_four",
    "noise_one",
    "noise_two",
    "noise_three",
    "noise_four"
]


odf = pd.DataFrame(data)
odf.head(5)

In [ ]:
std_df = odf.std().to_frame()
mean_df = odf.mean().to_frame()
std_df.columns = ["Std"]
mean_df.columns = ["Mean"]
merged_df = pd.merge(std_df, mean_df, left_index=True, right_index=True)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
ax.bar(
    [
        "M1",
        "M2",
        "M3",
        "M4",
        "X1",
        "X2",
        "X3",
        "X4",
        "C1",
        "C2",
        "C3",
        "C4",
        "Noise 1",
        "Noise 2",
        "Noise 3",
        "Noise 4",
    ],
    [
        std_df["Std"].loc["m_one"],
        std_df["Std"].loc["m_two"],
        std_df["Std"].loc["m_three"],
        std_df["Std"].loc["m_four"],
        std_df["Std"].loc["x_one"],
        std_df["Std"].loc["x_two"],
        std_df["Std"].loc["x_three"],
        std_df["Std"].loc["x_four"],
        std_df["Std"].loc["c_one"],
        std_df["Std"].loc["c_two"],
        std_df["Std"].loc["c_three"],
        std_df["Std"].loc["c_four"],
        std_df["Std"].loc["noise_one"],
        std_df["Std"].loc["noise_two"],
        std_df["Std"].loc["noise_three"],
        std_df["Std"].loc["noise_four"],
    ],
    color=[
        darker,
        darker,
        darker,
        darker,
        important,
        important,
        important,
        important,
        light,
        light,
        light,
        light,
        noise,
        noise,
        noise,
        noise,
    ],
)
ax.set_xlabel("Input Column and Node Number")
ax.set_ylabel("Standard Deviation")
;

In [ ]:
g_ms = sum(
    [
        std_df["Std"].loc["m_one"],
        std_df["Std"].loc["m_two"],
        std_df["Std"].loc["m_three"],
        std_df["Std"].loc["m_four"],
    ]
)
g_xs = sum(
    [
        std_df["Std"].loc["x_one"],
        std_df["Std"].loc["x_two"],
        std_df["Std"].loc["x_three"],
        std_df["Std"].loc["x_four"],
    ]
)
g_cs = sum(
    [
        std_df["Std"].loc["c_one"],
        std_df["Std"].loc["c_two"],
        std_df["Std"].loc["c_three"],
        std_df["Std"].loc["c_four"],
    ]
)
g_noises = sum(
    [
        std_df["Std"].loc["noise_one"],
        std_df["Std"].loc["noise_two"],
        std_df["Std"].loc["noise_three"],
        std_df["Std"].loc["noise_four"],
    ]
)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
ax.bar(
    ["Ms", "Xs", "Cs", "Noises"],
    [g_ms, g_xs, g_cs, g_noises],
    color=[darker, important, light, noise],
)
;

## Scatter Plots

In [ ]:
def prepare_scatter_df(data: List, bin: int, abs: bool = False):
    indices = [i for i in range(len(data))]
    df = pd.DataFrame(indices, columns=["Indices"])
    rdf = pd.DataFrame(data, columns=["Readings"])
    if abs:
        rdf = rdf.explode("Readings", ignore_index=True).abs()
    else:
        rdf = rdf.explode("Readings", ignore_index=True)
    df["Readings"] = rdf["Readings"]
    df["Bin"] = [bin for _ in range(len(data))]
    return df

In [ ]:
def get_group_by_node(index: int) -> List:
    group = []
    for i in list(history.keys()):
        group.append(history[i][index])
    return group

In [ ]:
def call_prepare_scatter_df(data: List, abs: bool = False):
    dfs = []
    for i, item in enumerate(data):
        dfs.append(prepare_scatter_df(item, i, abs))
    return pd.concat(dfs)

In [ ]:
def create_scatter(scatter_dfs):
    _, ax = plt.subplots(figsize=(16, 6))
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Weight")
    colormap = np.array([darker, important, light, noise])
    categories = np.array(scatter_dfs["Bin"])
    ax.scatter(
        x=scatter_dfs["Indices"],
        y=scatter_dfs["Readings"],
        c=colormap[categories],
        s=[2],
    )

In [ ]:
groups = [get_group_by_node(i) for i in range(len(history.keys()))]

In [ ]:
for group in groups:
    sdf = call_prepare_scatter_df(group, True)
    create_scatter(sdf)

## Probability Densities

In [ ]:
def plot_stds(group):
    _, ax = plt.subplots(figsize=(16,6))
    domain = np.linspace(-2,2,1000)
    means = [mean(group[0]), mean(group[1]), mean(group[2]), mean(group[3])]
    stds = [stdev(group[0]), stdev(group[1]), stdev(group[2]), stdev(group[3])]
    colors = [darker, important, light, noise]

    for mu, std, color in zip(means, stds, colors):
        probs = norm.pdf(domain, mu, std)
        ax.plot(domain, probs, color=color)
        ax.set_xlabel("Weight")
        ax.set_ylabel("Probability Density")

In [ ]:
for group in groups:
    plot_stds(group)

## Means, etc.

In [ ]:
def get_props(group):
    domain = np.linspace(-2,2,1000)
    means = [mean(group[0]), mean(group[1]), mean(group[2]), mean(group[3])]
    stds = [stdev(group[0]), stdev(group[1]), stdev(group[2]), stdev(group[3])]
    results = []
    for mu, std in zip(means, stds):
        values = []
        probs = norm.pdf(domain, mu, std)
        for prob in probs:
            if math.floor(prob) != 0:
                values.append(prob)
        results.append(values)
    return results

In [ ]:
all_probs = [get_props(group) for group in groups]

In [ ]:
def barrage(all_probs, nodes: int):
    res = {
        i: {"len": 0, "max": 0, "sum": 0, "mean": 0, "stdev": 0} for i in range(nodes)
    }
    for probs in all_probs:
        for i, prob in enumerate(probs):
            res[i]["len"] += len(prob)
            res[i]["max"] += max(prob)
            res[i]["sum"] += sum(prob)
            res[i]["mean"] += mean(prob)
            res[i]["stdev"] += stdev(prob)
    return res

In [ ]:
columns = ["ms", "xs", "cs", "noises"]

res = barrage(all_probs, len(columns))
for i in range(len(columns)):
    res[columns[i]] = res.pop(i)
df = pd.DataFrame(res)
df

In [ ]:
def plot_barrage(method: str):
    cdict = {"ms": light, "xs": important, "cs": darker, "noises": noise}
    data = pd.Series(df.loc[method], df.columns).sort_values(ascending=True)
    cmap = [cdict[i] for i in list(data.index.values)]
    ax = data.plot.barh(width=0.8, figsize=(3, 1), color=cmap)
    ax.set_ylabel("Input")
    ax.set_xlabel(method.capitalize())

In [ ]:
plot_barrage("len")

In [ ]:
plot_barrage("max")

In [ ]:
plot_barrage("sum")

In [ ]:
plot_barrage("mean")

In [ ]:
plot_barrage("stdev")